In [63]:
from scipy.optimize import linprog
import copy

In [21]:
wtp_matrix = [[5, 5, 5], [4, 5, 6], [7, 8, 9]]
price_matrix = [[5, 2, 3], [4, 5, 6], [7, 8, 9]]
feasability = [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
feasability_budget = [1, 1, 1]
budget_list = [10, 10, 10]

prob_matrix = []

demand_matrix = []
x_bounds = (0, 1)

In [22]:
wtp_temp = [[-x for x in row] for row in wtp_matrix]
print wtp_temp

[[-5, -5, -5], [-4, -5, -6], [-7, -8, -9]]


In [84]:
res = linprog(wtp_matrix[0], A_ub=price_matrix[0], b_ub=budget_list[0],
              A_eq=feasability, b_eq=feasability_budget, 
              bounds=(x_bounds, x_bounds, x_bounds), options={"disp": True}
             )

Optimization terminated successfully.
         Current function value: 5.000000    
         Iterations: 1


In [85]:
print res

     fun: 5.0
 message: 'Optimization terminated successfully.'
     nit: 1
   slack: array([ 5.,  0.,  1.,  1.])
  status: 0
 success: True
       x: array([ 1.,  0.,  0.])


In [57]:
# matrix of WTP -- subjects along rows, treatments along columns
wtp_matrix = [[5, 5, 5], [4, 5, 6], [7, 8, 9]]

# matrix of (alpha, beta) values -- alphas are in 1st row, betas in 2nd
ab_matrix = [[1, 1, 1], [2, 2, 2]]
# matrix of predicted treatment effect -- subjects along rows, treatments along columns
pred_treat_effect = [[1, 1, 1], [2, 2, 2], [3, 3, 3]]

# price_matrix = [[5, 2, 3], [4, 5, 6], [7, 8, 9]]

budget_list = [10, 10, 10]
capacity_list = [1, 1, 1]

In [83]:
def solve_demand_matrix(wtp_matrix, price_matrix, budget_list):
    # verify dimensions
    # same number of subjects
    assert(len(wtp_matrix) == len(price_matrix) == len(budget_list))
    # same number of treatments
    assert(len(wtp_matrix[0]) == len(price_matrix[0]))
    
    prob_matrix = []
    
    # for feasability condition
    feasability = [[1 for x in row] for row in wtp_matrix]
    feasability_budget = [1 for x in budget_list]
    x_bounds = (0, 1)
    
#     feasability = [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
#     feasability_budget = [1, 1, 1]
    
    # convert wtp_matrix to negative for maximization using linprog
    wtp_temp = [[-x for x in row] for row in wtp_matrix]
    
    # solve LP problem row by row
    for i, row in enumerate(wtp_temp):
        res = linprog(wtp_temp[i], A_ub=price_matrix[i], b_ub=budget_list[i],
                  A_eq=feasability, b_eq=feasability_budget, 
                  bounds=(x_bounds,)*len(wtp_temp[0]))
        prob_matrix.append(res.x.tolist())
    
    return prob_matrix

In [81]:
x_bounds = (0, 1)
print (x_bounds,)*3

((0, 1), (0, 1), (0, 1))


In [56]:
def solve_excess_demand(demand_matrix, capacity_list):
    # make sure the dimensions make sense
    # we have a capacity for each treatment
    assert (len(demand_matrix[0]) == len(capacity_list))
    
    # sum along the columns of demand_matrix to get total demand
    total_demand_list = [0]*len(capacity_list)
    for i in demand_matrix:
        for j, val in enumerate(i):
            total_demand_list[j] += val
    
    # subtract to get excess demand
    excess_demand = [x1 - x2 for x1, x2 in zip(total_demand_list, capacity_list)]
    return excess_demand

In [69]:
def solve_price_matrix(pred_treat_effect, ab_matrix):
    # verify dimensions -- need to have a, b for each treatment
    assert(len(pred_treat_effect[0]) == len(ab_matrix[0]) == len(ab_matrix[1]))
    
    price_matrix = copy.deepcopy(pred_treat_effect)
    
    # the price p(i,t) is a linear function of the input 
    for i in range(len(pred_treat_effect)):
        for j in range(len(pred_treat_effect[0])):
            price_matrix[i][j] = ab_matrix[0][j]*pred_treat_effect[i][j] + ab_matrix[1][j]
            
    return price_matrix

In [70]:
pred_treat_effect = [[1, 1, 1], [2, 2, 2], [3, 3, 3]]
ab_matrix = [[1, 1, 1], [2, 2, 2]]
print solve_price_matrix(pred_treat_effect, ab_matrix)

[[3, 3, 3], [4, 4, 4], [5, 5, 5]]


In [59]:
price_matrix = solve_price
demand_matrix = solve_demand(wtp_matrix, price_matrix, budget_list)
excess_demand = solve_excess_demand(demand_matrix, capacity_list)
print demand_matrix
print excess_demand

[[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 0.0, 1.0]]
[0.0, -1.0, 1.0]
